<a href="https://colab.research.google.com/github/karindrima/Auto-encoder/blob/main/Coding_round_Arindrima_Koley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installation of packages
!pip install pandas
!pip install datetime
import pandas as pd
import math

In [ ]:
#loading dataset
url = '/content/Coding_Pool.csv'
df1 = pd.read_csv(url)

In [ ]:
#Property occupancy adjustment factor is added as column factor1
df1.loc[df1['occupancy.type']== 'sh', 'factor1'] = 1.2
df1.loc[df1['occupancy.type']== 'btl', 'factor1'] = 1.3
df1.loc[df1['occupancy.type'].isin(['own','other','part']), 'factor1'] = 1

In [ ]:
#Standardising date timeframe
alsofdate = pd.to_datetime('2020-03-31')
df1['datecolumn']= pd.to_datetime(df1['int.switch.date'])
df1['days'] = (alsofdate - df1['datecolumn']).dt.days
df1['days.net.calc']= (df1['days']/365)* 12
referance_date = pd.to_datetime('1970-01-01')

In [ ]:
#Payment shock adjustment is saved in factor2 column
df1.loc[(df1['interest.product']== 'DISCOUNT')& (df1['datecolumn']!= referance_date) & (df1['days.net.calc']<= 6) ,
        'factor2'] = 1.5
df1.loc[(df1['interest.product']== 'DISCOUNT')& (df1['datecolumn']!= referance_date) & (df1['days.net.calc']> 6) ,
        'factor2'] = 1
df1.loc[(df1['interest.product']!= 'DISCOUNT'), 'factor2'] = 1

In [ ]:
num_distinct_categories = df1['region'].nunique() #to find the no of different regions
category_counts = df1['region'].value_counts()  #to get the count of each region
print(num_distinct_categories)
print(category_counts)
total_loans= df1.shape[0]
if total_loans> 1 & total_loans<= 250  :      #calculate small pool adjustment
  small_pool_adjust = 16.0839/ math.log(total_loans)
else :
  small_pool_adjust = 1
total_adjusted_bal = df1['adjusted.bal'].sum()  # total summation of the Adjusted balance
region_list = df1['region'].unique() #creating a list of unique regions to find geo.concentration
print(region_list)

12
london              79
east midlands       66
scotland            36
east anglia         20
north                4
north west           4
northern ireland     4
south east           4
south west           4
wales                4
west midlands        4
yorks and humber     4
Name: region, dtype: int64
['london' 'scotland' 'east midlands' 'east anglia' 'north' 'north west'
 'northern ireland' 'south east' 'south west' 'wales' 'west midlands'
 'yorks and humber']


In [ ]:
geo_concentration = {}
for i in region_list :    #for loop for calculating geo concentration of each region
  result = df1.loc[df1['region'] == i, 'adjusted.bal'].sum()
  geo_concentration[i]=result/ total_adjusted_bal * 100

print(geo_concentration)
selected_values={key: value for key, value in geo_concentration.items() if value> 20}
df1.loc[df1['region']== selected_values, 'factor3'] = 1.5 #Geo concentration adjustment is saved in factor3 column
df1.loc[df1['region']!= selected_values, 'factor3'] = 1

{'london': 34.089427125971795, 'scotland': 15.66285727516502, 'east midlands': 28.370652045079414, 'east anglia': 8.432019807929304, 'north': 1.6789795063450421, 'north west': 1.577349668212525, 'northern ireland': 1.6510332026145533, 'south east': 1.6912505866322818, 'south west': 1.7056896687060306, 'wales': 1.6340029780547347, 'west midlands': 1.767092614041054, 'yorks and humber': 1.739645521248248}


In [ ]:
df1['factor']= df1['factor1'] * df1['factor2'] * df1['factor3']
print(df1['factor'])
df1['ff_calc']= 12 * df1['factor']
df1.loc[df1['ff_calc']< 100, 'ff'] = df1['ff_calc']  #Loan level forclosure frequency of each loan is saved in ff column
df1.loc[df1['ff_calc']> 100, 'ff'] = 100 #If foreclosure frequency is greater than 100 then setting it to 100
print(df1['ff'])

0      1.0
1      1.5
2      1.0
3      1.0
4      1.0
      ... 
228    1.0
229    1.0
230    1.2
231    1.2
232    1.2
Name: factor, Length: 233, dtype: float64
0      12.0
1      18.0
2      12.0
3      12.0
4      12.0
       ... 
228    12.0
229    12.0
230    14.4
231    14.4
232    14.4
Name: ff, Length: 233, dtype: float64


In [ ]:
ff_adjusted_bal =(df1['adjusted.bal'] * df1['ff']).sum()
print(ff_adjusted_bal)
WAFF_calc = (ff_adjusted_bal/total_adjusted_bal)* (small_pool_adjust)
WAFF = min(100, WAFF_calc) #Weighted average foreclosure frequency
print(WAFF)
CC = WAFF * 40 # credit coverage with WALS= 40
print("Credit coverage =", CC)

140967306.0
39.55080318100446
Credit coverage = 1582.0321272401784
